In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
text_file = open("/kaggle/input/vocab12/file.txt", "r")
lines = text_file.readlines()
text_file.close()

In [3]:
type(lines)

list

In [4]:
vocab = list(set(lines))
type(vocab)

list

In [6]:
def delete_letter(word):
    return  [word[:i]+word[i+1:] for i in range(len(word))]

In [5]:
def switch_letter(word):
    switch_l = []

    for i in range(len(word)-1):
        w_l = re.findall('\w', word)
        if i-1 < 0:
            w_l[i:i+2] = w_l[i+1::-1]
        else:
            w_l[i:i+2] = w_l[i+1:i-1:-1]
            
        switch_l.append(''.join(w_l))
    
    return switch_l

In [13]:
def replace_letter(word):
    letters = 'ابتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإئ'
    
    replace_set = set()

    for i in range(len(word)):
        for l in letters:
            new_word = word[:i]+l+word[i+1:]
            if new_word == word:
                continue
            replace_set.add(new_word)

    replace_l = sorted(list(replace_set))
    
    return replace_l

In [14]:
def insert_letter(word):
    letters = 'ابتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإئ'
    insert_l = []

    for i in range(len(word)+1):
        for l in letters:
            new_word = word[:i]+l+word[i:]
            insert_l.append(new_word)
    
    return insert_l

In [15]:
def edit_one_letter(word, allow_switches = True):
    edit_one_set = delete_letter(word)+insert_letter(word)+replace_letter(word)

    if allow_switches:
        edit_one_set += switch_letter(word)

    return set(edit_one_set)

In [16]:
def edit_two_letters(word, allow_switches = True):
    edit_two_set = []
    edit_one_set = edit_one_letter(word)
    
    for edit in edit_one_set:
        edit_two_set += edit_one_letter(edit)
        
    return set(edit_two_set) | set(edit_one_set)

In [17]:
def get_corrections(word, vocab):
    suggestions = []
    
    correct_word_suggest = [word] if word in vocab else []
    edit_one_letter_suggest = list(filter(lambda item: item in vocab, list(edit_one_letter(word))))
    edit_two_letter_suggest = list(filter(lambda item: item in vocab, list(edit_two_letters(word))))
    
    suggestions = correct_word_suggest or edit_one_letter_suggest or edit_two_letter_suggest or ['لم يتم العثور علي إقتراحات مناسبة لهذه الكلمة']

    return set(suggestions) 

In [18]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    m = len(source) 
    n = len(target) 
    D = np.zeros((m+1, n+1), dtype=int) 
  
    for row in range(1, m+1):
        D[row,0] =  D[row-1,0]+del_cost
        
    for col in range(1, n+1):
        D[0,col] = D[0, col-1]+ins_cost
        
    for row in range(1, m+1):
        for col in range(1, n+1):
            r_cost = rep_cost

            if source[row-1] == target[col-1]:
                r_cost = 0
                
            D[row,col] = np.min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])

    med = D[m,n]
    
    return med

In [19]:
def get_suggestions(corrections, word):
  distance = []
  suggest = []
  
  for correction in corrections:
    source = word
    target = correction
    min_edits = min_edit_distance(source, target)

    distance.append(min_edits)
    suggest.append(correction)

  suggest_result = list(map(lambda idx: suggest[idx], np.argsort(distance))) 
  return suggest_result

In [20]:
def ar_spelling_checker(text):

  word_l = re.findall('\w{3,}', text)
  result = {}

  for word in word_l:
    tmp_corrections = []
    if not word in vocab[0]:
      tmp_corrections = get_corrections(word, vocab[0])
      if len(tmp_corrections) == 0:
        continue
      result[word] = get_suggestions(tmp_corrections, word)

  return result

In [21]:
word="خمستاشر"

In [22]:
edit_one_letter("فرضلك")

{'ءرضلك',
 'ءفرضلك',
 'آرضلك',
 'آفرضلك',
 'أرضلك',
 'أفرضلك',
 'ؤرضلك',
 'ؤفرضلك',
 'إرضلك',
 'إفرضلك',
 'ئرضلك',
 'ئفرضلك',
 'ارضلك',
 'افرضلك',
 'برضلك',
 'بفرضلك',
 'ةرضلك',
 'ةفرضلك',
 'ترضلك',
 'تفرضلك',
 'ثرضلك',
 'ثفرضلك',
 'جرضلك',
 'جفرضلك',
 'حرضلك',
 'حفرضلك',
 'خرضلك',
 'خفرضلك',
 'درضلك',
 'دفرضلك',
 'ذرضلك',
 'ذفرضلك',
 'ررضلك',
 'رضلك',
 'رفرضلك',
 'رفضلك',
 'زرضلك',
 'زفرضلك',
 'سرضلك',
 'سفرضلك',
 'شرضلك',
 'شفرضلك',
 'صرضلك',
 'صفرضلك',
 'ضرضلك',
 'ضفرضلك',
 'طرضلك',
 'طفرضلك',
 'ظرضلك',
 'ظفرضلك',
 'عرضلك',
 'عفرضلك',
 'غرضلك',
 'غفرضلك',
 'فءرضلك',
 'فءضلك',
 'فآرضلك',
 'فآضلك',
 'فأرضلك',
 'فأضلك',
 'فؤرضلك',
 'فؤضلك',
 'فإرضلك',
 'فإضلك',
 'فئرضلك',
 'فئضلك',
 'فارضلك',
 'فاضلك',
 'فبرضلك',
 'فبضلك',
 'فةرضلك',
 'فةضلك',
 'فترضلك',
 'فتضلك',
 'فثرضلك',
 'فثضلك',
 'فجرضلك',
 'فجضلك',
 'فحرضلك',
 'فحضلك',
 'فخرضلك',
 'فخضلك',
 'فدرضلك',
 'فدضلك',
 'فذرضلك',
 'فذضلك',
 'فرءضلك',
 'فرءلك',
 'فرآضلك',
 'فرآلك',
 'فرأضلك',
 'فرألك',
 'فرؤضلك',
 'فرؤلك',
 'فرإضلك',
 'فر

In [23]:
ar_spelling_checker("فرضلك")

{'فرضلك': ['فضلك']}

In [24]:
ar_spelling_checker("أنا خلصت البيزين ")

{'البيزين': ['البيسين', 'البنزين']}

In [25]:
cc=ar_spelling_checker("وحتة")
cc

{'وحتة': ['حتة',
  'وتة',
  'وحت',
  'وحة',
  'وحشة',
  'وشتة',
  'وستة',
  'لحتة',
  'وحوة',
  'وحتل',
  'وحتش']}

In [26]:
new_words = pd.read_excel("/kaggle/input/word-label/words_labels1 (3).xlsx")
new_words

,sentence,new_word,Tokenizer,Label
0,وحياه ربنا خايفه منك اكتير,اكتير,"['اكت', '##ير']",1
1,إتعلموا ثقافة السكوت يا جماعة إتعلموا تسكتوا ل...,تسكتوا,"['تسكت', '##وا']",1
2,زيك الرجاله هتنقرض,هتنقرض,"['هتن', '##قر', '##ض']",1
3,أنا هتعود و انتا كمان لازم تعود,هتعود,"['هتع', '##ود']",1
4,دا من وجهة نظر الى بيتاسف بس,بيتاسف,"['بيتا', '##سف']",1
...,...,...,...,...
60,يلا البسو بقى انتو الى زنيتو علينا,زنيتو,"['زن', '##يتو']",2
61,قاعدة بدور على عذر واحد كان يصبرنى على بروده م...,متغفلة,"['متغ', '##فلة']",2
62,قاعدة بدور على عذر واحد كان يصبرنى على بروده م...,تغفيلة,"['تغ', '##فيلة']",2
63,ميصحش كده في شيوع في الاتهام,شيوع,"['شي', '##وع']",2


In [27]:
new_words['correct']=new_words['new_word'].apply(lambda new: ar_spelling_checker(str(new)))

In [28]:
new_words

,sentence,new_word,Tokenizer,Label,correct
0,وحياه ربنا خايفه منك اكتير,اكتير,"['اكت', '##ير']",1,"{'اكتير': ['اتير', 'الكتير', 'اكتر', 'كتير', '..."
1,إتعلموا ثقافة السكوت يا جماعة إتعلموا تسكتوا ل...,تسكتوا,"['تسكت', '##وا']",1,"{'تسكتوا': ['ستوا', 'تستا', 'تستو', 'سكتنا', '..."
2,زيك الرجاله هتنقرض,هتنقرض,"['هتن', '##قر', '##ض']",1,{'هتنقرض': ['هنعرض']}
3,أنا هتعود و انتا كمان لازم تعود,هتعود,"['هتع', '##ود']",1,"{'هتعود': ['تعود', 'هتعد', 'متعود', 'اتعود']}"
4,دا من وجهة نظر الى بيتاسف بس,بيتاسف,"['بيتا', '##سف']",1,"{'بيتاسف': ['بيتس', 'بيتسم', 'بيتحس', 'بيلاف',..."
...,...,...,...,...,...
60,يلا البسو بقى انتو الى زنيتو علينا,زنيتو,"['زن', '##يتو']",2,{'زنيتو': ['زيتو']}
61,قاعدة بدور على عذر واحد كان يصبرنى على بروده م...,متغفلة,"['متغ', '##فلة']",2,"{'متغفلة': ['متبلة', 'مغسلة', 'متوفرة', 'منغزل..."
62,قاعدة بدور على عذر واحد كان يصبرنى على بروده م...,تغفيلة,"['تغ', '##فيلة']",2,"{'تغفيلة': ['تقيلة', 'تغطية', 'تشكيلة']}"
63,ميصحش كده في شيوع في الاتهام,شيوع,"['شي', '##وع']",2,"{'شيوع': ['شيو', 'شيوس', 'شروع', 'شيون']}"


In [29]:
new_novel = pd.read_excel("/kaggle/input/novveeelll/Novel_test.xlsx")
new_novel

,sentence,new_word,Tokenizer,Label
0,موضوع الإشاعة بتاعة ده لازم يتجاب سليم وحتة وا...,وحتة,"['وحت', '##ة']",1.0
1,كنت قاعد بتكلم انا وبابا فمره فحوار اني علي طو...,فمره,"['فمر', '##ه']",1.0
2,كنت قاعد بتكلم انا وبابا فمره فحوار اني علي طو...,فحوار,"['فح', '##وار']",1.0
3,اول مرة اطلب قهوة في قهوة فقالي سادة قولتلة ما...,فقالي,"['فقال', '##ي']",1.0
4,لحد البيت بتاعنا اتهد,اتهد,"['اته', '##د']",1.0
5,ماتديش أي حد حاجه,ماتديش,"['ماتد', '##يش']",1.0
6,ال انا مافرقش معاهم هما برده مايفرقوش معايا,مافرقش,"['مافر', '##قش']",1.0
7,مفيش زكرا هترجعني,زكرا,"['زك', '##را']",1.0
8,تفسح شويه فى المجلس,تفسح,"['تفس', '##ح']",1.0
9,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,مروتى,"['مرو', '##تى']",1.0


In [30]:
new_novel['correct']=new_words['new_word'].apply(lambda neww: ar_spelling_checker(str(neww)))

In [31]:
new_novel

,sentence,new_word,Tokenizer,Label,correct
0,موضوع الإشاعة بتاعة ده لازم يتجاب سليم وحتة وا...,وحتة,"['وحت', '##ة']",1.0,"{'اكتير': ['اتير', 'الكتير', 'اكتر', 'كتير', '..."
1,كنت قاعد بتكلم انا وبابا فمره فحوار اني علي طو...,فمره,"['فمر', '##ه']",1.0,"{'تسكتوا': ['ستوا', 'تستا', 'تستو', 'سكتنا', '..."
2,كنت قاعد بتكلم انا وبابا فمره فحوار اني علي طو...,فحوار,"['فح', '##وار']",1.0,{'هتنقرض': ['هنعرض']}
3,اول مرة اطلب قهوة في قهوة فقالي سادة قولتلة ما...,فقالي,"['فقال', '##ي']",1.0,"{'هتعود': ['تعود', 'هتعد', 'متعود', 'اتعود']}"
4,لحد البيت بتاعنا اتهد,اتهد,"['اته', '##د']",1.0,"{'بيتاسف': ['بيتس', 'بيتسم', 'بيتحس', 'بيلاف',..."
5,ماتديش أي حد حاجه,ماتديش,"['ماتد', '##يش']",1.0,{'بيفهمها': ['بيفهموا']}
6,ال انا مافرقش معاهم هما برده مايفرقوش معايا,مافرقش,"['مافر', '##قش']",1.0,"{'فعالم': ['فالم', 'عالم', 'فعال', 'لعالم', 'ف..."
7,مفيش زكرا هترجعني,زكرا,"['زك', '##را']",1.0,{'فخيالي': ['خيالي']}
8,تفسح شويه فى المجلس,تفسح,"['تفس', '##ح']",1.0,"{'بيجمعني': ['بيجمع', 'بيعني']}"
9,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,مروتى,"['مرو', '##تى']",1.0,"{'محتجلك': ['محتاجل', 'محلك', 'محتاجله', 'منتج..."
